# import など準備

In [435]:
# import
import sys
import os
import re
import numpy as np
import pandas as pd
from robobrowser import RoboBrowser
# from robobrowser.browser import RoboState
from retry import retry
from dateutil.parser import parse
from datetime import datetime

In [210]:
# 比較参照用
kabupro = pd.read_csv('/Users/Really/Stockyard/_csv/kessan_table.csv')

In [211]:
kabupro

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,2.230000e+08,2.13,1.96,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-03
1,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2.265000e+09,21.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
2,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
3,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2.806000e+09,26.72,24.63,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
4,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2.433000e+09,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
6,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05
7,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
8,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
9,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11


In [34]:
# 比較参照用
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日'],
      dtype='object')

In [48]:
kabupro.ix[14]
# 株プロにしか無い項目: 希薄化後一株当り純利益, 純資産又は株主資本, 営業キャッシュフロー, 投資キャッシュフロー, 財務キャッシュフロー

証券コード                      1301
企業名                          極洋
会計基準                       日本基準
連結個別                         連結
決算期                    2017年3月期
決算期間                         通期
期首                   2016-04-01
期末                   2017-03-31
名寄前勘定科目（売上高欄に掲載）            売上高
売上高                 2.36561e+11
営業利益                  3.723e+09
経常利益                  3.709e+09
純利益                   2.422e+09
一株当り純利益                  230.66
希薄化後一株当り純利益              213.01
純資産又は株主資本            2.5391e+10
総資産                  9.7391e+10
一株当り純資産                 2378.09
営業キャッシュフロー             6.01e+08
投資キャッシュフロー           -1.998e+09
財務キャッシュフロー             1.05e+08
情報公開又は更新日            2017-05-11
Name: 14, dtype: object

## 参考コード

In [ ]:
def get_price_yahoojp(code, start=None, end=None, interval='d'): # start = '2017-01-01'
    # http://sinhrks.hatenablog.com/entry/2015/02/04/002258
    # http://jbclub.xii.jp/?p=598
    base = 'http://info.finance.yahoo.co.jp/history/?code={0}.T&{1}&{2}&tm={3}&p={4}'
    
    start = pd.to_datetime(start) # Timestamp('2017-01-01 00:00:00')

    if end == None:
        end = pd.to_datetime(pd.datetime.now())
    else :
        end = pd.to_datetime(end)
    start = 'sy={0}&sm={1}&sd={2}'.format(start.year, start.month, start.day) # 'sy=2017&sm=1&sd=1'
    end = 'ey={0}&em={1}&ed={2}'.format(end.year, end.month, end.day)
    p = 1
    tmp_result = []

    if interval not in ['d', 'w', 'm', 'v']:
        raise ValueError("Invalid interval: valid values are 'd', 'w', 'm' and 'v'")

    while True:
        url = base.format(code, start, end, interval, p)
        # print(url)
        # https://info.finance.yahoo.co.jp/history/?code=7203.T&sy=2000&sm=1&sd=1&ey=2017&em=10&ed=13&tm=d&p=1
        tables = get_table(url)
        if len(tables) < 2 or len(tables[1]) == 0:
            # print('break')
            break
        tmp_result.append(tables[1]) # ページ内の3つのテーブルのうち2番目のテーブルを連結
        p += 1
        # print(p)
        
    result = pd.concat(tmp_result, ignore_index=True) # インデックスをゼロから振り直す

    result.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'AdjClose'] # 列名を変更
    if interval == 'm':
        result['Date'] = pd.to_datetime(result['Date'], format='%Y年%m月')
    else:
        result['Date'] = pd.to_datetime(result['Date'], format='%Y年%m月%d日') # 日付の表記を変更
    result = result.set_index('Date') # インデックスを日付に変更
    result = result.sort_index()
    
    stock_name = tables[0].columns[0]
    # print([code, stock_name])
    
    return [result, stock_name]

In [ ]:
%%writefile amazon_order_history.py

# Amazon.co.jpの注文履歴を取得する

import sys
import os
from robobrowser import RoboBrowser

# 認証の情報は環境変数から取得する
AMAZON_EMAIL = os.environ['AMAZON_EMAIL']
AMAZON_PASSWORD = os.environ['AMAZON_PASSWORD']

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')


def main():
    # 注文履歴のページを開く
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')
    
    # サインインページにリダイレクトされていることを確認する
    assert 'Amazonサインイン' in browser.parsed.title.string
    
    # name="signIn" というサインインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form(attrs={'name': 'signIn'})
    form['email'] = AMAZON_EMAIL
    form['password'] = AMAZON_PASSWORD
    
    # フォームを送信する。正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要。
    print('Signing in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer': browser.url,
        'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    })
    
    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())

    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string # 注文履歴画面が表示されていることを確認する。
        
        print_order_history() # 注文履歴を表示する。
        
        link_to_next = browser.get_link('次へ') #「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break #「次へ」のリンクがない場合はループを抜けて終了する。
            
        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next) # 次へ」というリンクをたどる。
        
        
def print_order_history():
    """
    現在のページのすべての注文履歴を表示する
    """
    for line_item in browser.select('.order-info'):
        order = {} # 注文の情報を格納するためのdict
        # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる
        # 注文の情報のすべての列について反復する
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value
        print(order['注文日'], order['合計']) # 注文の情報を表示する。
        

if __name__ == '__main__':
    main()

In [ ]:
!forego run python amazon_order_history.py

## get_table 関数 (retry 付き pd.read_html)

In [19]:
@retry(tries=5, delay=1, backoff=2)
def get_table(url):
    result = pd.read_html(url, header=0) # header引数で0行目をヘッダーに指定。データフレーム型
    
    return result

# データの取得 (スクリプト版未完成)

In [49]:
%%writefile kabutan_get_table.py

import sys
import os
from robobrowser import RoboBrowser
from robobrowser import RoboState
import numpy as np
import pandas as pd

# 認証の情報は環境変数から取得する
KT_EMAIL = os.environ['KT_EMAIL']
KT_PASSWORD = os.environ['KT_PASSWORD']

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')

# ログインページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://account.kabutan.jp/login')

# 株探プレミアムページにいることを確認する
assert '株探プレミアム' in browser.parsed.title.string

# name="signIn" というサインインフォームを埋める。
# フォームのname属性の値はブラウザーの開発者ツールで確認できる。
form = browser.get_form(action='/login')
form['session[email]'] = KT_EMAIL
form['session[password]'] = KT_PASSWORD

# フォームを送信する。
# 正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要な場合がある。
print('Signing in...', file=sys.stderr)
browser.submit_form(form)
'''
browser.submit_form(form, headers={
    'Referer': browser.url,
    'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
})
'''

# 個別銘柄の決算ページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://kabutan.jp/stock/finance?code=7203&mode=k')

# 決算ページにいることを確認する
assert '決算' in browser.parsed.title.string

# ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
# print(browser.parsed.prettify())

# print(browser.find())
# print(browser.select('table'))
# print(browser.select('div.fin_fd:nth-child(7) > table:nth-child(1) > tbody:nth-child(1)'))

#kessan = browser.find()
kessan = browser.select('table')
print(kessan)

#result = pd.read_html(kessan, flavor='lxml') #, header=0
#print(result)

#print('success')

'''
def main():
    # 注文履歴のページを開く
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')
    
    # サインインページにリダイレクトされていることを確認する
    assert 'Amazonサインイン' in browser.parsed.title.string
    
    # name="signIn" というサインインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form(attrs={'name': 'signIn'})
    form['email'] = KT_EMAIL
    form['password'] = KT_PASSWORD
    
    # フォームを送信する。正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要。
    print('Signing in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer': browser.url,
        'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    })
    
    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())

    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string # 注文履歴画面が表示されていることを確認する。
        
        print_order_history() # 注文履歴を表示する。
        
        link_to_next = browser.get_link('次へ') #「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break #「次へ」のリンクがない場合はループを抜けて終了する。
            
        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next) # 次へ」というリンクをたどる。
        
        
def print_order_history():
    """
    現在のページのすべての注文履歴を表示する
    """
    for line_item in browser.select('.order-info'):
        order = {} # 注文の情報を格納するためのdict
        # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる
        # 注文の情報のすべての列について反復する
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value
        print(order['注文日'], order['合計']) # 注文の情報を表示する。
        

if __name__ == '__main__':
    main()
'''

Overwriting kabutan_get_table.py


In [52]:
kessan = !forego run python kabutan_get_table.py
kessan

# ログイン。入力したメアドとパスワードが出力に表示されてしまうのでそのままでGitHubに上げちゃダメ！

In [349]:
# sign-in
# 認証の情報
KT_EMAIL = input('Name?')
KT_PASSWORD = input('Password?')

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')

# ログインページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://account.kabutan.jp/login')

# 株探プレミアムページにいることを確認する
assert '株探プレミアム' in browser.parsed.title.string

# name="signIn" というサインインフォームを埋める。
# フォームのname属性の値はブラウザーの開発者ツールで確認できる。
form = browser.get_form(action='/login')
form['session[email]'] = KT_EMAIL
form['session[password]'] = KT_PASSWORD

# フォームを送信する。
# 正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要な場合がある。
print('Signing in...', file=sys.stderr)
browser.submit_form(form)

print(browser.select('.is-success')[0].text.strip())

Name?hideshi.honma@gmail.com
Password?reallyKT1173


Navigating...
Signing in...


ログインしました


# 個別銘柄の決算ページを開く

In [350]:
# 個別銘柄の決算ページを開く
code = 7203 # トヨタ

print('Navigating...', file=sys.stderr)
browser.open('https://kabutan.jp/stock/finance?code={0}&mode=k'.format(code))

# 決算ページにいることを確認する
assert '決算' in browser.parsed.title.string

print(browser.select('.kobetsu_data_table1_meigara')[0].text.strip())

Navigating...


トヨタ自動車


## ページ全体

In [351]:
kessan_html = browser.find()
kessan_html

<html lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<title>トヨタ自動車 (トヨタ) 【7203】：決算/業績・財務推移 [通期/半期/四半期] | 株探</title>
<meta content="トヨタ自動車 の直近5期の業績推移表、独自の8四半期(3ヵ月毎)の業績推移表、3期の財務データ推移表など豊富な決算情報が満載！ 経常利益の進捗率、採算性がわかる売上営業利益率、有利子負債倍率などプロ仕様のデータで、アナリスト並みの業績・財務分析ができます。トヨタ自動車 による決算発表・業績修正はリアルタイムで更新。業績推移表は、売上高 営業利益 経常利益 最終利益(純利益) 1株利益 1株配当 決算発表日 を表記。財務データは 1株純資産 自己資本比率 総資産 純資産 剰余金 有利子負債倍率 を表記。四半期累計や半期(上期・下期)の業績推移表も掲載しています。" name="description"/>
<meta content="トヨタ,トヨタ自動車,とよたじどうしゃ,とよた,TOYOTA MOTOR,7203,輸送用機器,株式,株価,投資,決算,業績推移,決算発表,業績修正,通期,四半期,3ヵ月,四半期累計,上期,上半期,下期,下半期,進捗率,進捗度,売上営業利益率,売上高,営業利益,経常利益,最終利益,純利益,1株利益,1株配当,決算発表日,1株純資産,自己資本比率,株主資本比率,総資産,純資産,自己資本,株主資本,剰余金,有利子負債倍率" name="keywords"/>
<meta content="9TyMQp63vRE7zxgZJofMB6CJlyYQBwnVoHJoVFo1_rg" name="google-site-verification"/>
<meta c

In [377]:
tables = pd.read_html(str(kessan_html), header=0)
tables[11]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,連 1999.03,12749008.0,774945.0,771885.0,356180.0,94.2,23,99/05/24
2,連 2000.03,12879561.0,775982.0,797058.0,406798.0,110.0,24,00/05/17
3,連 2001.03,13424423.0,870131.0,972273.0,471295.0,127.9,25,01/05/16
4,連 2002.03,15106297.0,1123470.0,1113524.0,615824.0,170.7,28,02/05/13
5,U 2003.03,15501553.0,1271646.0,1226652.0,750942.0,211.3,36,03/05/08
6,U 2004.03,17294760.0,1666890.0,1765793.0,1162098.0,342.9,45,04/05/11
7,U 2005.03,18551526.0,1672187.0,1754637.0,1171260.0,355.4,65,05/05/10
8,U 2006.03,21036909.0,1878342.0,2087360.0,1372180.0,421.8,90,06/05/10
9,U 2007.03,23948091.0,2238683.0,2382516.0,1644032.0,512.1,120,07/05/09


In [188]:
# 後でhtml形式で読み込み可能なファイルとして書き出す方法
kabutan_kessan = open('kabutan_kessan.html', 'w')
kabutan_kessan.write(str(browser.find()))
kabutan_kessan.close()

In [191]:
# 保存したhtmlファイルからの読み込み
tables = pd.read_html('/Users/Really/Stockyard/kabutan_kessan.html', header=0)

## ページ内のテーブル属性のみ

In [133]:
tables = pd.read_html(str(browser.select('table')), header=0)
tables[12]

Navigating...


,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
# 後でhtml形式で読み込み可能なファイルとして書き出す方法
kabutan_kessan_tables = open('kabutan_kessan_tables.html', 'w')
kabutan_kessan_tables.write(str(browser.select('table')))
kabutan_kessan_tables.close()

In [109]:
# 保存したhtmlファイルからの読み込み
tables = pd.read_html('/Users/Really/Stockyard/kessan_tables.html', header=0)

## リスト、テーブルの概要

In [192]:
len(tables)

29

In [136]:
len(tables[12])

52

In [140]:
len(tables[12].columns)

14

## tables[7] 銘柄概要

In [193]:
tables[7]

,7203,トヨタ自動車,東証１,15:00


##  tables[9] 銘柄概要

In [194]:
tables[9]

,業績：
0,輸送用機器
1,単位 100株


## tables [10] ＰＥＲ ＰＢＲ 利回り 信用倍率

In [195]:
tables[10]

,ＰＥＲ,ＰＢＲ,利回り,信用倍率
0,10.4倍,1.12倍,－％,1.14倍


## tables[11] 通期業績

In [217]:
kabupro[kabupro['証券コード'] == 7203]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
38471,7203,トヨタ自動車,米国基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,6.390688e+12,...,5.877740e+11,185.43,185.34,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-04
38472,7203,トヨタ自動車,米国基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.294560e+13,...,1.126836e+12,356.08,355.91,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-05
38473,7203,トヨタ自動車,米国基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,2.011561e+13,...,1.726863e+12,545.94,545.68,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
38474,7203,トヨタ自動車,米国基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.723452e+13,...,2.173338e+12,688.02,687.66,1.764733e+13,4.772983e+13,5334.96,3.685753e+12,-3.813490e+12,3.060450e+11,2016-05-11
38475,7203,トヨタ自動車,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,1.120941e+13,...,1.690679e+12,535.22,534.95,1.018427e+13,1.512862e+13,3235.44,NaN,NaN,NaN,2016-05-11
38476,7203,トヨタ自動車,米国基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,6.987648e+12,...,6.463940e+11,205.41,205.30,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-04
38477,7203,トヨタ自動車,米国基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.409149e+13,...,1.258112e+12,399.39,397.75,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-08
38478,7203,トヨタ自動車,米国基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,2.143137e+13,...,1.886077e+12,601.44,597.29,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-06
38479,7203,トヨタ自動車,米国基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.840312e+13,...,2.312694e+12,741.36,735.36,1.808819e+13,4.742760e+13,5513.08,4.460857e+12,-3.182544e+12,-4.235710e+11,2017-05-10
38480,7203,トヨタ自動車,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,1.158582e+13,...,1.810370e+12,581.08,575.64,1.085944e+13,1.610021e+13,3417.07,NaN,NaN,NaN,2017-05-10


In [196]:
tables[11]
# 株プロに無い項目: １株配

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,連 1999.03,12749008.0,774945.0,771885.0,356180.0,94.2,23,99/05/24
2,連 2000.03,12879561.0,775982.0,797058.0,406798.0,110.0,24,00/05/17
3,連 2001.03,13424423.0,870131.0,972273.0,471295.0,127.9,25,01/05/16
4,連 2002.03,15106297.0,1123470.0,1113524.0,615824.0,170.7,28,02/05/13
5,U 2003.03,15501553.0,1271646.0,1226652.0,750942.0,211.3,36,03/05/08
6,U 2004.03,17294760.0,1666890.0,1765793.0,1162098.0,342.9,45,04/05/11
7,U 2005.03,18551526.0,1672187.0,1754637.0,1171260.0,355.4,65,05/05/10
8,U 2006.03,21036909.0,1878342.0,2087360.0,1372180.0,421.8,90,06/05/10
9,U 2007.03,23948091.0,2238683.0,2382516.0,1644032.0,512.1,120,07/05/09


In [378]:
# 全ての列項目がnullの行を除去
tables[11] = tables[11][~tables[11].isnull().all(axis=1)].reset_index(drop=True)

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,連 1999.03,12749008.0,774945.0,771885.0,356180.0,94.2,23,99/05/24
1,連 2000.03,12879561.0,775982.0,797058.0,406798.0,110.0,24,00/05/17
2,連 2001.03,13424423.0,870131.0,972273.0,471295.0,127.9,25,01/05/16
3,連 2002.03,15106297.0,1123470.0,1113524.0,615824.0,170.7,28,02/05/13
4,U 2003.03,15501553.0,1271646.0,1226652.0,750942.0,211.3,36,03/05/08
5,U 2004.03,17294760.0,1666890.0,1765793.0,1162098.0,342.9,45,04/05/11
6,U 2005.03,18551526.0,1672187.0,1754637.0,1171260.0,355.4,65,05/05/10
7,U 2006.03,21036909.0,1878342.0,2087360.0,1372180.0,421.8,90,06/05/10
8,U 2007.03,23948091.0,2238683.0,2382516.0,1644032.0,512.1,120,07/05/09
9,U 2008.03,26289240.0,2270375.0,2437222.0,1717879.0,540.6,140,08/05/08


In [379]:
# 予想値と前期比の行を除去
tables[11] = tables[11][~((tables[11]['決算期'].str.contains('予')) | (tables[11]['決算期'].str.contains('前期比')))].reset_index(drop=True)

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,連 1999.03,12749008.0,774945.0,771885.0,356180.0,94.2,23,99/05/24
1,連 2000.03,12879561.0,775982.0,797058.0,406798.0,110.0,24,00/05/17
2,連 2001.03,13424423.0,870131.0,972273.0,471295.0,127.9,25,01/05/16
3,連 2002.03,15106297.0,1123470.0,1113524.0,615824.0,170.7,28,02/05/13
4,U 2003.03,15501553.0,1271646.0,1226652.0,750942.0,211.3,36,03/05/08
5,U 2004.03,17294760.0,1666890.0,1765793.0,1162098.0,342.9,45,04/05/11
6,U 2005.03,18551526.0,1672187.0,1754637.0,1171260.0,355.4,65,05/05/10
7,U 2006.03,21036909.0,1878342.0,2087360.0,1372180.0,421.8,90,06/05/10
8,U 2007.03,23948091.0,2238683.0,2382516.0,1644032.0,512.1,120,07/05/09
9,U 2008.03,26289240.0,2270375.0,2437222.0,1717879.0,540.6,140,08/05/08


In [454]:
# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[11][['会計基準', '決算期']] = pd.DataFrame(list(tables[11]['決算期'].str.split(' ')))

In [455]:
# 列の並び替え
tables[11] = tables[11][['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日']]

In [456]:
# 100万円単位換算
tables[11][['売上高', '営業益', '経常益', '最終益']] = tables[11][['売上高', '営業益', '経常益', '最終益']].apply(lambda x: x * 1000000)

In [457]:
# 型変換
tables[11]['１株配'] = tables[11]['１株配'].astype(float)

In [458]:
# 日付のパース、datetime.dateへの型変換
# tables[11]['決算期'] = tables[11]['決算期'].apply(lambda x: datetime.strptime(x, '%Y.%m').date()) # 日付ではないので文字列のままの方がいいかも？
tables[11]['発表日'] = tables[11]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[11]['発表日'] = pd.to_datetime(tables[11]['発表日'], format='%Y-%m-%d')
# tables[11]['決算期'] = pd.to_datetime(tables[11]['決算期'], format='%Y-%m-%d')

In [459]:
tables[11]

,会計基準,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,連,1999.03,1.274901e+13,7.749450e+11,7.718850e+11,3.561800e+11,94.2,23.0,1999-05-24
1,連,2000.03,1.287956e+13,7.759820e+11,7.970580e+11,4.067980e+11,110.0,24.0,2000-05-17
2,連,2001.03,1.342442e+13,8.701310e+11,9.722730e+11,4.712950e+11,127.9,25.0,2001-05-16
3,連,2002.03,1.510630e+13,1.123470e+12,1.113524e+12,6.158240e+11,170.7,28.0,2002-05-13
4,U,2003.03,1.550155e+13,1.271646e+12,1.226652e+12,7.509420e+11,211.3,36.0,2003-05-08
5,U,2004.03,1.729476e+13,1.666890e+12,1.765793e+12,1.162098e+12,342.9,45.0,2004-05-11
6,U,2005.03,1.855153e+13,1.672187e+12,1.754637e+12,1.171260e+12,355.4,65.0,2005-05-10
7,U,2006.03,2.103691e+13,1.878342e+12,2.087360e+12,1.372180e+12,421.8,90.0,2006-05-10
8,U,2007.03,2.394809e+13,2.238683e+12,2.382516e+12,1.644032e+12,512.1,120.0,2007-05-09
9,U,2008.03,2.628924e+13,2.270375e+12,2.437222e+12,1.717879e+12,540.6,140.0,2008-05-08


In [463]:
tables[11].dtypes

会計基準            object
決算期             object
売上高            float64
営業益            float64
経常益            float64
最終益            float64
１株益            float64
１株配            float64
発表日     datetime64[ns]
dtype: object

## tables[12] 業績予想

In [197]:
tables[12].columns

Index(['決算期', '修正日', '修正方向', '売上高', '営業益', '経常益', '最終益', '修正配当', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13'],
      dtype='object')

In [198]:
tables[12].columns = ['会計基準', '決算期', '修正日', 
                                   '結合修正方向', '売上高修正方向', '営業益修正方向', '経常益修正方向', '最終益修正方向', '修正配当修正方向', 
                                   '売上高', '営業益', '経常益', '最終益', '修正配当',]

In [199]:
tables[12][tables[12].index % 2 == 0].reset_index(drop=True)

,会計基準,決算期,修正日,結合修正方向,売上高修正方向,営業益修正方向,経常益修正方向,最終益修正方向,修正配当修正方向,売上高,営業益,経常益,最終益,修正配当
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
2,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
3,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
4,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
5,U,NaN,13/08/02,↑↑↑↑‐,↑,↑,↑,↑,‐,24000000.0,1940000.0,2030000.0,1480000.0,－
6,U,NaN,13/11/06,↑↑↑↑‐,↑,↑,↑,↑,‐,25000000.0,2200000.0,2290000.0,1670000.0,－
7,U,NaN,14/02/04,↑↑↑↑‐,↑,↑,↑,↑,‐,25500000.0,2400000.0,2530000.0,1900000.0,－
8,U,実績,14/05/08,↑↓↓↓‐,↑,↓,↓,↓,‐,25691911.0,2292112.0,2441080.0,1823119.0,165
9,U,2015.03,14/05/08,NaN,NaN,NaN,NaN,NaN,NaN,25700000.0,2300000.0,2390000.0,1780000.0,－


In [200]:
# 不要行
tables[12][tables[12].index % 2 != 0]

,会計基準,決算期,修正日,結合修正方向,売上高修正方向,営業益修正方向,経常益修正方向,最終益修正方向,修正配当修正方向,売上高,営業益,経常益,最終益,修正配当
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,↑,↓,↓,↓,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## tables[23] 過去最高 【実績】

In [201]:
tables[23]

,Unnamed: 0,売上高,営業益,経常益,最終益,１株益
0,過去最高,28403118.00,2853971.00,2983381.00,2312694.00,741.40
1,決算期,2016.03,2016.03,2016.03,2016.03,2016.03


## tables[24] 下期業績

In [202]:
tables[24]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,U 14.10-03,14288924.0,1398618.0,1383654.0,1046502.0,331.3,125,15/05/08
1,U 15.10-03,14311630.0,1270566.0,1308230.0,1054582.0,338.1,110,16/05/11
2,U 16.10-03,14526660.0,877507.0,1017291.0,884936.0,292.6,110,17/05/10
3,U 予 17.10-03,14308793.0,903459.0,997827.0,878672.0,298.6,－,17/11/07
4,前年同期比,-1.5,3.0,-1.9,-0.7,2.1,NaN,(%)


## tables[25] 第２四半期累計決算【実績】

In [203]:
tables[25]

,決算期,売上高,営業益,経常益,最終益,１株益,対通期進捗率,発表日
0,U 15.04-09,14091488.0,1583405.0,1675151.0,1258112.0,399.4,56.1,15/11/05
1,U 16.04-09,13070533.0,1116865.0,1176534.0,946173.0,311.1,53.6,16/11/08
2,U 17.04-09,14191207.0,1096541.0,1252173.0,1071328.0,359.6,55.7,17/11/07
3,前年同期比,8.6,-1.8,6.4,13.2,15.6,NaN,(%)


## tables[26] ３ヵ月業績の推移【実績】 累積ではなく差分？

In [204]:
tables[26]

,決算期,売上高,営業益,経常益,最終益,１株益,売上営業損益率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U 12.10-12,5318752.0,124757.0,131249.0,99914.0,31.6,2.3,13/02/05
2,U 13.01-03,5837086.0,502381.0,477863.0,313980.0,99.1,8.6,13/05/08
3,U 13.04-06,6255319.0,663383.0,724163.0,562194.0,177.4,10.6,13/08/02
4,U 13.07-09,6282166.0,592092.0,619362.0,438429.0,138.4,9.4,13/11/06
5,U 13.10-12,6585044.0,600509.0,678956.0,525464.0,165.8,9.1,14/02/04
6,U 14.01-03,6569382.0,436128.0,418599.0,297032.0,93.7,6.6,14/05/08
7,U 14.04-06,6390688.0,692728.0,771826.0,587774.0,185.4,10.8,14/08/05
8,U 14.07-09,6554909.0,659218.0,737348.0,539062.0,170.3,10.1,14/11/05
9,U 14.10-12,7170017.0,762880.0,846461.0,600027.0,189.7,10.6,15/02/04


## tables[27] 財務 【実績】

In [205]:
tables[27]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,連 1998.03,1583.20,43.5,13854355.0,6021895.0,5213340.0,－,98/08/01
2,連 1999.03,1642.90,41.9,14753311.0,6175937.0,5367879.0,－,90/08/01
3,連 2000.03,1841.50,41.3,16469054.0,6796666.0,6014125.0,0.64,00/05/17
4,連 2001.03,1956.57,40.6,17519427.0,7114567.0,6162656.0,0.63,01/05/16
5,連 2002.03,2059.94,36.8,19888937.0,7325072.0,6527956.0,0.79,02/05/13
6,U 2003.03,2063.43,35.3,20152974.0,7121000.0,7219896.0,1.02,03/05/08
7,U 2004.03,2456.08,37.1,22040228.0,8178567.0,8326215.0,0.92,04/05/11
8,U 2005.03,2767.67,37.2,24335011.0,9044950.0,9332176.0,0.95,05/05/10
9,U 2006.03,3257.63,36.8,28731595.0,10560449.0,10459788.0,0.98,06/05/10


In [183]:
tables[29]

IndexError: list index out of range

# テーブル内容の確認 (未ログイン)

In [6]:
url = 'https://kabutan.jp/stock/finance?code=1301&mode=k#kakosaikou_zisseki'

tables = get_table(url)

In [8]:
len(tables)

22

In [31]:
# ページ上部の主要指標情報 1-4の各項目と同じところっぽい
tables[0]

,"日経平均 (11:30) 22,404.85 +53.73 0.24%",日経平均 (11:30),"22,404.85",+53.73,0.24%,米ドル円 (11:58) 112.40 -0.84 -0.74%,米ドル円 (11:58),112.40,-0.84,-0.74%,"ＮＹダウ (終値) 23,458.36 +187.08 0.80%",ＮＹダウ (終値),"23,458.36",+187.08,0.80%,"上海総合 (終値) 3,399.87 -0.79 -0.02%",上海総合 (終値),"3,399.87",-0.79,-0.02%
0,日経平均 (11:30),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22404.85,53.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.24%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,米ドル円 (11:58),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,112.40,-0.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.74%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ＮＹダウ (終値),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23458.36,187.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.80%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,上海総合 (終値),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 主要指標情報 日経平均
tables[1]

,日経平均 (11:30),Unnamed: 1
0,22404.85,53.73
1,0.24%,NaN


In [10]:
# 主要指標情報 米ドル円
tables[2]

,米ドル円 (11:58),Unnamed: 1
0,112.40,-0.84
1,-0.74%,NaN


In [11]:
# 主要指標情報 ＮＹダウ (終値)
tables[3]

,ＮＹダウ (終値),Unnamed: 1
0,23458.36,187.08
1,0.80%,NaN


In [12]:
# 主要指標情報 上海総合 (終値)
tables[4]

,上海総合 (終値),Unnamed: 1
0,3399.87,-0.79
1,-0.02%,NaN


In [13]:
# 検索窓
tables[5]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,「株探」で配信したニュース記事を検索します,Unnamed: 5


In [14]:
# 銘柄概要
tables[6]

,1301 極洋 東証１ 11:29,1301,極洋,東証１,11:29,Unnamed: 5
0,1301,極洋,東証１,11:29,NaN,NaN
1,"3,645円 前日比 -30 (-0.82%) 比較される銘柄： マルハニチロ 、日水 、カネコ種","3,645円",前日比,-30,(-0.82%),比較される銘柄： マルハニチロ 、日水 、カネコ種
2,"3,645円",前日比,-30,(-0.82%),NaN,NaN
3,比較される銘柄： マルハニチロ 、日水 、カネコ種,NaN,NaN,NaN,NaN,NaN


In [15]:
# 銘柄概要
tables[7]

,1301,極洋,東証１,11:29


In [16]:
# 銘柄概要
tables[8]

,"3,645円",前日比,-30,(-0.82%)
0,比較される銘柄： マルハニチロ 、日水 、カネコ種,NaN,NaN,NaN


In [17]:
# 銘柄概要
tables[9]

,業績：
0,水産・農林業
1,単位 100株


In [18]:
# ＰＥＲ ＰＢＲ 利回り 信用倍率
tables[10]

,ＰＥＲ,ＰＢＲ,利回り,信用倍率
0,14.2倍,1.46倍,1.37％,1.80倍


In [19]:
# 通期業績
tables[11]
# 株プロに無い項目: １株配

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014.03,202387.0,2915.0,2985.0,2968.0,28.3,5.0,14/05/09
2,2015.03,218350.0,2460.0,2107.0,2433.0,23.2,5.0,15/05/08
3,2016.03,226626.0,2433.0,2814.0,1799.0,17.1,5.0,16/05/09
4,2017.03,236561.0,3723.0,3709.0,2422.0,230.7,60.0,17/05/11
5,予 2018.03,250000.0,4000.0,4000.0,2700.0,257.0,50.0,17/05/11
6,前期比,5.7,7.4,7.8,11.5,11.4,NaN,(%)
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 業績予想修正履歴
tables[12]

,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,2013.03,12/05/11,NaN,NaN,NaN,NaN,NaN,NaN,185000.0,3200.0,3100.0,1800.0,50.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,12/10/26,↓↓↓↓→,↓,↓,↓,↓,→,175000.0,2000.0,2200.0,1100.0,50.0
3,↓,↓,↓,↓,→,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,実績,13/05/10,↑↑↑↑→,↑,↑,↑,↑,→,178046.0,2324.0,2262.0,1269.0,50.0
5,↑,↑,↑,↑,→,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2014.03,13/05/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,13/10/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,実績,14/05/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2015.03,14/05/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 不明
tables[13]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4


In [22]:
# 修正履歴修正方向
tables[14]

,↓,↓.1,↓.2,↓.3,→


In [23]:
# 修正履歴修正方向
tables[15]

,↑,↑.1,↑.2,↑.3,→


In [24]:
# 過去最高 【実績】
tables[16]

,Unnamed: 0,売上高,営業益,経常益,最終益,１株益
0,過去最高,236561.00,3723.00,3709.00,5540.0,230.70
1,決算期,2017.03,2017.03,2017.03,1985.1,2017.03


In [25]:
# 下期業績
tables[17]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,14.10-03,115874.0,1739.0,1421.0,168.0,1.6,5.0,15/05/08
1,15.10-03,118127.0,1307.0,1407.0,888.0,8.5,5.0,16/05/09
2,16.10-03,126991.0,2552.0,2705.0,1236.0,117.7,60.0,17/05/11
3,予 17.10-03,129542.0,1754.0,1604.0,1067.0,101.6,50.0,17/11/06
4,前年同期比,2.0,-31.3,-40.7,-13.7,-13.7,NaN,(%)


In [26]:
# 第２四半期累計決算【実績】
tables[18]

,決算期,売上高,営業益,経常益,最終益,１株益,対通期進捗率,発表日
0,15.04-09,108499.0,1126.0,1407,911.0,8.7,50.0,15/11/06
1,16.04-09,109570.0,1171.0,1004,1186.0,113.0,27.1,16/11/04
2,17.04-09,120458.0,2246.0,2396,1633.0,155.5,59.9,17/11/06
3,前年同期比,9.9,91.8,2.4倍,37.7,37.6,NaN,(%)


In [27]:
# ３ヵ月業績の推移【実績】
# 累積ではなく差分？
tables[19]

,決算期,売上高,営業益,経常益,最終益,１株益,売上営業損益率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15.10-12,70391.0,1341.0,1281,222.0,2.1,1.9,16/02/05
2,16.01-03,47736.0,-34.0,126,666.0,6.3,-0.1,16/05/09
3,16.04-06,52206.0,467.0,380,551.0,5.2,0.9,16/08/05
4,16.07-09,57364.0,704.0,624,635.0,60.5,1.2,16/11/04
5,16.10-12,70405.0,1701.0,1823,1263.0,120.3,2.4,17/02/10
6,17.01-03,56586.0,851.0,882,-27.0,-2.6,1.5,17/05/11
7,17.04-06,56844.0,979.0,1103,754.0,71.8,1.7,17/08/04
8,17.07-09,63614.0,1267.0,1293,879.0,83.7,2.0,17/11/06
9,前年同期比,10.9,80.0,2.1倍,38.4,38.3,NaN,(%)


In [28]:
# 財務 【実績】
tables[20]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015.03,215.65,25.5,88937.0,22649.0,16537.0,2.09,15/05/08
2,2016.03,214.97,23.9,94608.0,22578.0,17739.0,2.24,16/05/09
3,2017.03,2378.09,25.6,97391.0,24976.0,19637.0,2.04,17/05/11
4,17.04-09,－,22.0,119806.0,26312.0,20640.0,2.50,17/11/06
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 日経平均チャート切り替え
tables[21]

,Unnamed: 0,日中足
0,日 足,NaN
